In [4]:
import pandas as pd
import matplotlib.pyplot  as plt
import os

In [5]:
dirname = os.path.join(os.path.abspath(os.curdir), 'data')

tsdf_b = pd.read_csv(os.path.join(os.sep, dirname, 'gemini_BTCUSD_2020_1min.csv'))
tsdf_e = pd.read_csv(os.path.join(os.sep, dirname,  'gemini_ETHUSD_2020_1min.csv'))
tsdf_l = pd.read_csv(os.path.join(os.sep, dirname, 'gemini_LTCUSD_2020_1min.csv'))
raw_data = (tsdf_b, tsdf_e, tsdf_l)